In [1]:
import pandas as pd
import numpy as np

In [2]:
for i in range(5):
    dft = pd.read_csv(f"../Assignment 3/query{i+1}/rank.csv")
    df = pd.read_csv(f"query{i+1}/rank.csv")

    df[["vsm_rank","vsm_score","bm25_rank","bm25_score"]] = dft[["vsm_rank","vsm_score","bm25_rank","bm25_score"]] 
    df.to_csv(f"query{i+1}/rank.csv", index=False)

In [63]:
idx = 5

In [67]:
df = pd.read_csv(f"query{idx}/rank.csv", encoding_errors='ignore')

In [68]:
def change_val(row):
    return 0 if row in [-1, -2] else 1
    

def calc_kappa(df, num_way = 5):
    if num_way == 2:
        df["annotation1"] = df["annotation1"].apply(lambda x: change_val(x))
        df["annotation2"] = df["annotation2"].apply(lambda x: change_val(x))

    p_a = len(df[df["annotation1"] == df["annotation2"]]) / len(df)
    p_e = 0
    for v in df["annotation1"].unique():
        p_e += (df["annotation1"].value_counts().get(v,0) / len(df)) * (df["annotation2"].value_counts().get(v,0) / len(df))

    kappa = (p_a - p_e) / (1 - p_e)
    
    return kappa

kappa_5 = calc_kappa(df)
kappa_2 = calc_kappa(df, 2)

In [70]:
kappa_5

0.07766990291262138

In [54]:
df = pd.read_csv(f"query{idx}/rank.csv",encoding_errors='ignore')

In [55]:
df["annotation"] = df[["annotation1", "annotation2"]].mean(axis=1)
df["relevance"] = df["annotation"].apply(lambda x: 1 if x > 0 else 0)

In [56]:
map_list = ["vsm", "bm25"] + df["SE"].unique().tolist()
map_res = {}
for i, item in enumerate(map_list):
    map_val = 1
    recalls = []
    precisions = []
    if i > 1:
        dft = df[df["SE"] == item]
    else:
        dft = df.sort_values(by=[f"{item}_rank"], ascending=True)

    num_total_rel = len(dft[dft["relevance"] == 1])

    for j in range(len(dft)):
        dftt = dft.head(j+1)
        num_rel = len(dftt[dftt["relevance"] == 1])
        recalls.append(num_rel / num_total_rel)
        precisions.append(num_rel / (j+1))

    max_precisions = [0] * 11
    for j in range(11):
        p_cur = max([p for p, r in zip(precisions, recalls) if r >= j/10]+ [0])
        max_precisions[j] = max(max_precisions[j], p_cur)

    map_res[f"MAP-{item}"] = sum(max_precisions) / 11 

In [57]:
map_res

{'MAP-vsm': 0.9339712918660287,
 'MAP-bm25': 0.9518341307814993,
 'MAP-Bing': 0.7474747474747475,
 'MAP-Google': 1.0}

In [58]:
def change_values_ndcg(val):
    if val >= 2:
        return 3
    elif val > 0:
        return 2
    elif val > -1:
        return 1
    else:
        return 0

df["annotation"] = df["annotation"] = df[["annotation1", "annotation2"]].mean(axis=1)
df["annotation"] = df["annotation"].apply(lambda x: change_values_ndcg(x))

In [59]:
ndcg_list = ["vsm", "bm25"] + df["SE"].unique().tolist()
ndcg_res = {}
for i in [5, 10]:
    for j, item in enumerate(ndcg_list):
        if j > 1:
            dft = df[df["SE"] == item].sort_values(by=["rank"], ascending=True).head(i)
        else:
            dft = dft.sort_values(by=[f"{item}_rank"], ascending=True).head(i)

        annots = dft["annotation"].to_list()
        ranks = dft["rank"].to_list()
        dcg = sum([annots[k] / (np.log2(ranks[k]) if ranks[k] > 1 else 1) for k in range(len(dft))])
        
        df_perfect = dft.sort_values(by=["annotation"], ascending=False)[:i]
        p_score = sum([df_perfect["annotation"].iloc[k] / (np.log2(k+1) if k > 0 else 1) for k in range(len(df_perfect))])

        ndcg = dcg / p_score

        ndcg_res[f"NDGC@{i}-{item}"] = ndcg

In [60]:
ndcg_res

{'NDGC@5-vsm': 0.7786245500245952,
 'NDGC@5-bm25': 0.7786245500245952,
 'NDGC@5-Bing': 1.0,
 'NDGC@5-Google': 1.0,
 'NDGC@10-vsm': 0.9999999999999998,
 'NDGC@10-bm25': 1.0,
 'NDGC@10-Bing': 0.8934961773206895,
 'NDGC@10-Google': 0.9916153951844069}

In [61]:
measures = ["kappa2", "kappa5"] + list(map_res.keys()) + list(ndcg_res.keys())
scores = [kappa_2, kappa_5] + list(map_res.values()) + list(ndcg_res.values())

df = pd.DataFrame({"Measure": measures, "Score": scores})
df.to_csv(f"query{idx}/evaluation.csv", index=False)

In [62]:
df

,Measure,Score
0,kappa2,0.875000
1,kappa5,0.503546
2,MAP-vsm,0.933971
3,MAP-bm25,0.951834
4,MAP-Bing,0.747475
5,MAP-Google,1.000000
6,NDGC@5-vsm,0.778625
7,NDGC@5-bm25,0.778625
8,NDGC@5-Bing,1.000000
9,NDGC@5-Google,1.000000


In [74]:
res = {}
for i in range(5):
    df = pd.read_csv(f"query{i+1}/evaluation.csv")
    r = dict(zip(df.Measure, df.Score))
    for k,v in r.items():
        res[k] = (res.get(k, 0) + v) /2

In [75]:
res

{'kappa2': 0.6402660472972974,
 'kappa5': 0.3232419171060195,
 'MAP-vsm': 0.8723175230020238,
 'MAP-bm25': 0.8975141551696577,
 'MAP-Google': 0.8446969696969697,
 'MAP-Bing': 0.8205176767676768,
 'NDGC@5-vsm': 0.712693967913187,
 'NDGC@5-bm25': 0.712693967913187,
 'NDGC@5-Google': 0.8693688034527621,
 'NDGC@5-Bing': 0.9505956073983777,
 'NDGC@10-vsm': 0.960439396428318,
 'NDGC@10-bm25': 0.9604393964283181,
 'NDGC@10-Google': 0.859836631642539,
 'NDGC@10-Bing': 0.892955345302622,
 'MAP-Baidu': 0.5,
 'NDGC@5-Baidu': 0.5,
 'NDGC@10-Baidu': 0.4934556873605576,
 'MAP-Ecosia': 0.4636363636363637,
 'NDGC@5-Ecosia': 0.4796148122065969,
 'NDGC@10-Ecosia': 0.4782408454621405}